In [26]:
import pandas as pd
import numpy as np
from pandas.core.interchange.dataframe_protocol import DataFrame
from pygments.lexer import bygroups

import volpy_func_lib as vp
import load_clean_lib
import importlib
from datetime import datetime
import matplotlib.pyplot as plt

from volpy_func_lib import load_forward_price

In [27]:
importlib.reload(vp)
importlib.reload(load_clean_lib)

<module 'load_clean_lib' from 'D:\\Professionelt\\Code\\volpy\\load_clean_lib.py'>

# Lav Data

## Load, Clean and Prepare data

In [28]:
# Load data
profile = "Mads"
dataset = "i4s4"
od, FW, ZCY_curves = load_clean_lib.load_od_FW_ZCY(profile, dataset)


# first_day = pd.to_datetime("1996-01-04")
# last_day = pd.to_datetime("1997-01-01")
first_day = pd.to_datetime("1996-01-04")
# first_day = pd.to_datetime("2002-01-04")
last_day = pd.to_datetime("2003-02-28")
# last_day = pd.to_datetime("1996-05-01")

# Clean data
od = load_clean_lib.clean_od(od, first_day = first_day, last_day = last_day)

### Add FW to options

In [29]:
od = vp.add_FW_to_od(od, FW)

### remove if ITM

In [30]:
od = od.loc[
    ((od["F"] < od["K"]) & (od["cp_flag"] == "C")) | ((od["F"] > od["K"]) & (od["cp_flag"] == "P"))
]

### Add r to options

In [31]:
od = vp.add_r_to_od(od, ZCY_curves)

## Add low/high and create summary (Filters dataset for criteria such as min 3 strikes ... min 8 days...)

In [32]:
# Process each (date, ticker) group and collect summary information
summary_list = []
processed_groups = []
for (current_date, current_ticker), group in od.groupby(["date", "ticker"]):
    proc_group, summary = vp.process_group_activity_summary(group)
    summary["date"] = current_date
    summary["ticker"] = current_ticker
    summary_list.append(summary)
    processed_groups.append(proc_group)

# Recombine all groups back into the od DataFrame
od = pd.concat(processed_groups)

# Update summary_dly_df (indexed by (ticker, date)) using the gathered summary information
summary_df = pd.DataFrame(summary_list).set_index(["ticker", "date"])

# Make dataset for results for each day for each asset
summary_dly_df = load_clean_lib.summary_dly_df_creator(od)
summary_dly_df.update(summary_df)

### only keep the lowest and the second lowest ("high") TTMs

In [33]:
od_rdy = od[(od["low"] == True) | (od["high"] == True)]

## Calculate swap rate

In [34]:
importlib.reload(vp)
importlib.reload(load_clean_lib)

<module 'load_clean_lib' from 'D:\\Professionelt\\Code\\volpy\\load_clean_lib.py'>

In [37]:
summary_dly_df = vp.fill_swap_rates(summary_dly_df, od_rdy, n_points=200)

Processing Groups: 100%|██████████| 26686/26686 [13:30<00:00, 32.91it/s]


## Save as CSV

In [38]:
summary_dly_df.to_csv("data/summary_dly_df_v2.csv")
od_rdy.to_csv("data/od_rdy_v2.csv")

In [ ]:
summary_dly_df.to_csv("data/summary_dly_df.csv")
od_rdy.to_csv("data/od_rdy.csv")

# Load CSV

In [5]:
summary_dly_df = pd.read_csv("data/summary_dly_df.csv")
od_rdy = pd.read_csv("data/od_rdy.csv")

In [6]:
summary_dly_df

,Ticker,Date,#days,low days,high days,low #K,high #K,low SW,high SW,Active,Inactive reason
0,GE,1996-01-04,4.0,16.0,44.0,2.0,4.0,NaN,NaN,False,unique(K) < 3
1,GE,1996-01-05,4.0,15.0,43.0,2.0,4.0,NaN,NaN,False,unique(K) < 3
2,GE,1996-01-08,4.0,12.0,40.0,3.0,4.0,10.755724,6.849246,True,NaN
3,GE,1996-01-09,4.0,11.0,39.0,2.0,4.0,NaN,NaN,False,unique(K) < 3
4,GE,1996-01-10,4.0,10.0,38.0,3.0,4.0,10.606864,7.341032,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...
14403,AMZN,2003-02-24,5.0,26.0,54.0,4.0,7.0,33.356329,49.808785,True,NaN
14404,AMZN,2003-02-25,5.0,25.0,53.0,5.0,5.0,45.297606,40.968534,True,NaN
14405,AMZN,2003-02-26,5.0,24.0,52.0,5.0,6.0,44.528532,40.919518,True,NaN
14406,AMZN,2003-02-27,5.0,23.0,51.0,4.0,6.0,31.209627,38.491318,True,NaN


### Keep only active days

In [10]:
filtered_df = summary_dly_df[~summary_dly_df["low SW"].isna()]

### Full period analysis

In [51]:
T = 30
t = 0
T1 = summary_dly_df["low days"]
T2 = summary_dly_df["high days"]
SW1 = summary_dly_df["low SW"]
SW2 = summary_dly_df["high SW"]

summary_dly_df["SW"] = (1/(T-t)) * (SW1 * (T1-t) * (T2-T) + SW2 * (T2 - t) * (T - T1)) / (T2 - T1) 

In [52]:
import numpy as np

# Select only the numeric columns in the DataFrame.
numeric_cols = summary_dly_df.select_dtypes(include=[np.number]).columns

# Filter rows where "low SW" is not NaN, group by "Ticker", and calculate the mean for numeric columns.
avg_by_ticker = summary_dly_df[~summary_dly_df["low SW"].isna()].groupby("Ticker")[numeric_cols].mean()
avg_by_ticker

,#days,low days,high days,low #K,high #K,low SW,high SW,SW
Ticker,,,,,,,,
AMZN,4.033307,23.758128,62.854877,11.880254,15.962728,1.003961,0.978818,1.015770
DJX,5.850295,23.362832,53.561947,17.850295,18.008850,0.065105,0.067175,0.065260
GE,4.591778,23.635785,61.432542,5.697742,7.723798,0.127088,0.121009,0.124122
IBM,4.106229,23.354839,62.072859,9.722469,12.868743,0.163164,0.151368,0.159224
MSFT,4.105673,23.356507,62.078420,8.860400,12.284205,0.186210,0.179498,0.184226
NDX,4.696835,23.337590,53.949473,37.039423,36.615769,0.176860,0.172974,0.175492
OEX,4.479178,23.337590,53.588007,33.555802,30.267074,0.060215,0.061305,0.060745
SPX,5.829539,23.337590,53.603554,33.886174,29.960022,0.058055,0.058551,0.058166


In [61]:
avg_by_ticker

,#days,low days,high days,low #K,high #K,low SW,high SW,SW
Ticker,,,,,,,,
AMZN,4.033307,23.758128,62.854877,11.880254,15.962728,1.003961,0.978818,1.015770
DJX,5.850295,23.362832,53.561947,17.850295,18.008850,0.065105,0.067175,0.065260
GE,4.591778,23.635785,61.432542,5.697742,7.723798,0.127088,0.121009,0.124122
IBM,4.106229,23.354839,62.072859,9.722469,12.868743,0.163164,0.151368,0.159224
MSFT,4.105673,23.356507,62.078420,8.860400,12.284205,0.186210,0.179498,0.184226
NDX,4.696835,23.337590,53.949473,37.039423,36.615769,0.176860,0.172974,0.175492
OEX,4.479178,23.337590,53.588007,33.555802,30.267074,0.060215,0.061305,0.060745
SPX,5.829539,23.337590,53.603554,33.886174,29.960022,0.058055,0.058551,0.058166


In [59]:
summary_dly_df[~summary_dly_df["low SW"].isna()].groupby("Ticker")[numeric_cols].count()["SW"]

Ticker
AMZN    1261
DJX     1356
GE      1727
IBM     1798
MSFT    1798
NDX     1801
OEX     1801
SPX     1801
Name: SW, dtype: int64

In [54]:
avg_by_ticker[["SW", "low SW", "high SW"]]*100

,SW,low SW,high SW
Ticker,,,
AMZN,101.577048,100.396141,97.881762
DJX,6.526043,6.510473,6.717516
GE,12.412184,12.708765,12.100932
IBM,15.922442,16.316446,15.136809
MSFT,18.422565,18.620974,17.949796
NDX,17.549235,17.686020,17.297400
OEX,6.074466,6.021474,6.130478
SPX,5.816576,5.805479,5.855129


In [13]:
avg_by_ticker = summary_dly_df[~summary_dly_df["low SW"].isna()].groupby("Ticker").mean()

TypeError: agg function failed [how->mean,dtype->object]

In [23]:
summary_dly_df.shape

(14408, 11)

In [22]:
summary_dly_df[~summary_dly_df["low SW"].isna()]

,Ticker,Date,#days,low days,high days,low #K,high #K,low SW,high SW,Active,Inactive reason
2,GE,1996-01-08,4.0,12.0,40.0,3.0,4.0,10.755724,6.849246,True,NaN
4,GE,1996-01-10,4.0,10.0,38.0,3.0,4.0,10.606864,7.341032,True,NaN
5,GE,1996-01-11,4.0,9.0,37.0,3.0,4.0,9.021875,7.082273,True,NaN
6,GE,1996-01-12,4.0,36.0,64.0,4.0,6.0,7.163207,8.143591,True,NaN
7,GE,1996-01-15,3.0,33.0,61.0,4.0,6.0,7.511532,8.254066,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...
14403,AMZN,2003-02-24,5.0,26.0,54.0,4.0,7.0,33.356329,49.808785,True,NaN
14404,AMZN,2003-02-25,5.0,25.0,53.0,5.0,5.0,45.297606,40.968534,True,NaN
14405,AMZN,2003-02-26,5.0,24.0,52.0,5.0,6.0,44.528532,40.919518,True,NaN
14406,AMZN,2003-02-27,5.0,23.0,51.0,4.0,6.0,31.209627,38.491318,True,NaN


In [14]:
avg_by_ticker = summary_dly_df[~summary_dly_df["low SW"].isna()].groupby("Ticker").mean()
print(avg_by_ticker)


TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
avg_by_ticker

In [12]:
for ticker, group in grouped:
    print(f"Ticker: {ticker}")
    print(group)

Ticker: AMZN
      Ticker        Date  #days  low days  high days  low #K  high #K  \
13084   AMZN  1997-11-20    4.0      30.0       58.0     4.0      4.0   
13085   AMZN  1997-11-21    4.0      29.0       57.0     4.0      5.0   
13086   AMZN  1997-11-24    4.0      26.0       54.0     4.0      5.0   
13087   AMZN  1997-11-25    4.0      25.0       53.0     6.0      5.0   
13088   AMZN  1997-11-26    4.0      24.0       52.0     3.0      4.0   
...      ...         ...    ...       ...        ...     ...      ...   
14403   AMZN  2003-02-24    5.0      26.0       54.0     4.0      7.0   
14404   AMZN  2003-02-25    5.0      25.0       53.0     5.0      5.0   
14405   AMZN  2003-02-26    5.0      24.0       52.0     5.0      6.0   
14406   AMZN  2003-02-27    5.0      23.0       51.0     4.0      6.0   
14407   AMZN  2003-02-28    5.0      22.0       50.0     3.0      6.0   

           low SW     high SW  Active Inactive reason  
13084  117.016026  156.312842    True             NaN 

# Done, rest is random stuff before clean ect.

# Be ready for the D ata

# Replicating option IV's

In [24]:
od

,secid,date,exdate,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,ticker,n_trading_day,days,spread,F,r,low,high
0,105169,1996-01-04,1996-01-20,C,60.0,12.6250,13.1250,18,0.431174,GE,0,16,0.5000,72.874078,0.057209,True,False
1,105169,1996-01-04,1996-01-20,C,75.0,0.5000,0.5625,753,0.216178,GE,0,16,0.0625,72.874078,0.057209,True,False
2,105169,1996-01-04,1996-01-20,C,65.0,7.8750,8.0000,30,0.320320,GE,0,16,0.1250,72.874078,0.057209,True,False
3,105169,1996-01-04,1996-01-20,C,47.5,25.1250,25.6250,4,0.948733,GE,0,16,0.5000,72.874078,0.057209,True,False
4,105169,1996-01-04,1996-01-20,P,70.0,0.3125,0.3750,476,0.219093,GE,0,16,0.0625,72.874078,0.057209,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59264,108105,1996-05-01,1997-03-22,C,600.0,83.1250,84.1250,0,0.168632,SPX,82,325,1.0000,674.460867,0.057582,False,False
59265,108105,1996-05-01,1997-03-22,P,675.0,34.5000,35.5000,0,0.144289,SPX,82,325,1.0000,674.460867,0.057582,False,False
59266,108105,1996-05-01,1997-03-22,C,525.0,145.7500,146.7500,0,0.196454,SPX,82,325,1.0000,674.460867,0.057582,False,False
59267,108105,1996-05-01,1997-03-22,C,475.0,190.8750,191.8750,0,0.215242,SPX,82,325,1.0000,674.460867,0.057582,False,False


In [23]:
summary_dly_df

SwapRate  #days  low days  high days  low #K  high #K  \
Ticker Date                                                                
GE     1996-01-04       NaN    4.0      16.0       44.0     9.0      4.0   
       1996-01-05       NaN    4.0      15.0       43.0     5.0      4.0   
       1996-01-08       NaN    4.0      12.0       40.0     4.0      4.0   
       1996-01-09       NaN    4.0      11.0       39.0     3.0      4.0   
       1996-01-10       NaN    4.0      10.0       38.0     8.0      4.0   
...                     ...    ...       ...        ...     ...      ...   
SPX    1996-04-25       NaN    6.0      23.0       58.0    25.0     35.0   
       1996-04-26       NaN    6.0      22.0       57.0    26.0     35.0   
       1996-04-29       NaN    6.0      19.0       54.0    26.0     35.0   
       1996-04-30       NaN    6.0      18.0       53.0    25.0     34.0   
       1996-05-01       NaN    6.0      17.0       52.0    25.0     34.0   

                   Active Inactive reason  
Ticker Date                                
GE     1996-01-04    True                  
       1996-01-05    True                  
       1996-01-08    True                  
       1996-01-09    True                  
       1996-01-10    True                  
...                   ...             ...  
SPX    1996-04-25    True                  
       1996-04-26    True                  
       1996-04-29    True                  
       1996-04-30    True                  
       1996-05-01    True                  

[498 rows x 8 columns]

In [156]:
importlib.reload(vp)
importlib.reload(load_clean_lib)
import sys
import time

# record the code duration
start_time = time.time()

# loop through all dates
for current_date in od["date"].unique():
    # select current date data
    od_date = od[od["date"] == current_date]
    tickers_date = od_date["ticker"].unique()

    # loop through all tickers for the date
    for current_ticker in tickers_date:    
        # select current ticker date data
        od_ticker = od_date[od_date["ticker"] == current_ticker]

        # two functions to get and set info in summary_dly_df
        def get(info):
            return summary_dly_df.at[(current_ticker, current_date), info]        
        def set(info, value, reason = None):
            summary_dly_df.at[(current_ticker, current_date), info] = value
            if reason is not None:
                summary_dly_df.at[(current_ticker, current_date), "Inactive reason"] = reason

        days_ticker = od_ticker["days"].unique()
        set("#days", value = len(days_ticker)) # report number of unique dates for ticker on date
        
        # remove tickers that has less than two unique dates
        if len(days_ticker) < 2:
            set("Active", value = False, reason = "unique(_days) < 2") # False
            # print(f"skipped {current_ticker} for {current_date} due to no data available.")
            continue

        # remove tickers that has no dates within 90 days til expiration        
        if min(days_ticker) > 90:
            set("Active", value = False, reason = "min days > 90") # False
            print(f"skipped {current_ticker} for {current_date} due to min(days) > 90")
            continue
        
        # select two shortest dates that are above 7 days (skip first only if multiple)
        days_ticker = sorted(days_ticker)
        low_2_days = days_ticker[:2]
        if low_2_days[0] <= 8:
            if len(days_ticker) < 3:
                set("Active", value = False, reason = "min days <= 8 & len < 3") # False
                continue
            low_2_days = days_ticker[1:3]
        
        # report two selected days
        set("low days", value = low_2_days[0])
        set("high days", value = low_2_days[1])
        
        # report current date to be active for current ticker, if at least 3 unique strikes. Otherwise inactive
        set("Active", value = True, reason = "")
        for days, high_low in zip(low_2_days, ["low", "high"]): # loops through the highest and lowest days til expiration. e.g. 15 days ("low") and 32 days ("high").
            number_of_K = (od_ticker[od_ticker["days"] == days]["strike_price"]).nunique()
            set(f"{high_low} #K", value = number_of_K)
            if number_of_K < 3:
                set("Active", value = False, reason = f"unique(K) < 3")
        if not get("Active"):
            continue   # if not active
        
        # loop through the two selected dates
        for days in low_2_days:
            TTM = days / 365.0
            # if days > 100: break

            # select TTM data (only the given days til expiration)
            od_TTM = od_ticker[od_ticker["days"] == days]

            # IV = T_data["IV"]["impl_volatility"] #smile, would need to be interpolated            
            # K = T_data["od"]["strike_price"]
            # IV = T_data["od"]["impl_volatility"]
            # cp_flag = T_data["od"]["cp_flag"]
            # option_prices = vp.BSM(F, K, days, IV, r, cp_flag)

            # original slice where days match (idx: od_TTM -> od)
            idx = od_TTM.index

            # modify the original DataFrame in place
            od.loc[idx, "IV_option_price"] = od_TTM.apply(
                lambda row: vp.BSM(
                    row["F"],
                    row["strike_price"],
                    TTM,
                    row["impl_volatility"],
                    row["r"],
                    row["cp_flag"]
                ),
                axis=1
            )
            od.loc[idx, "moneyness"] = np.log10(F / od_TTM["strike_price"])
            
            
    # Print progress
    elapsed_time = time.time() - start_time
    sys.stdout.write(f"\rRunning date: {current_date} | Tickers: {len(tickers_date)} | Time elapsed: {elapsed_time:.2f} seconds")
    sys.stdout.flush()

Running date: 2000-07-03 00:00:00 | Tickers: 8 | Time elapsed: 24.50 seconds

KeyboardInterrupt: 

In [124]:
importlib.reload(vp)
importlib.reload(load_clean_lib)
import sys
import time

# record the code duration
start_time = time.time()

# loop through all dates
for current_date in od["date"].unique():
    # Current day asset data
    
    # select current date data
    od_date = od[od["date"] == current_date]
    FW_date = FW[FW["date"] == current_date]
    ZCY_date = ZCY_curves[ZCY_curves['date'] == current_date]

    tickers_date = od_date["ticker"].unique()

    # Current day Zero Cupon curve
    if ZCY_date.empty:
        print(f"No yield data available: {current_date} (interpolating)")
        ZCY_date = vp.interpolate_yield_curve_between_dates(ZCY_curves, current_date)

    # loop through all tickers for the date
    for current_ticker in tickers_date:        
        # two functions to get and set info in summary_dly_df
        def get(info):
            return summary_dly_df.at[(current_ticker, current_date), info]        
        def set(info, value, reason = None):
            summary_dly_df.at[(current_ticker, current_date), info] = value
            if reason is not None:
                summary_dly_df.at[(current_ticker, current_date), "Inactive reason"] = reason

        # select current ticker date data
        od_ticker = od_date[od_date["ticker"] == current_ticker]
        FW_ticker = FW_date[FW_date["ticker"] == current_ticker]
        days_ticker = od_ticker["days"].unique()
        set("#days", value = len(days_ticker)) # report number of unique dates for ticker on date
        
        # remove tickers that has less than two unique dates
        if len(days_ticker) < 2:
            set("Active", value = False, reason = "unique(_days) < 2") # False
            # print(f"skipped {current_ticker} for {current_date} due to no data available.")
            continue

        # remove tickers that has no dates within 90 days til expiration        
        if min(days_ticker) > 90:
            set("Active", value = False, reason = "min days > 90") # False
            print(f"skipped {current_ticker} for {current_date} due to min(days) > 90")
            continue
        
        # select two shortest dates that are above 7 days (skip first only if multiple)
        days_ticker = sorted(days_ticker)
        low_2_days = days_ticker[:2]
        if low_2_days[0] <= 8:
            if len(days_ticker) < 3:
                set("Active", value = False, reason = "min days <= 8 & len < 3") # False
                continue
            low_2_days = days_ticker[1:3]
        
        # report two selected days
        set("low days", value = low_2_days[0])
        set("high days", value = low_2_days[1])
        
        # report current date to be active for current ticker, if at least 3 unique strikes. Otherwise inactive
        set("Active", value = True, reason = "")
        for days, high_low in zip(low_2_days, ["low", "high"]): # loops through the highest and lowest days til expiration. e.g. 15 days ("low") and 32 days ("high").
            number_of_K = (od_ticker[od_ticker["days"] == days]["strike_price"]).nunique()
            set(f"{high_low} #K", value = number_of_K)
            if number_of_K < 3:
                set("Active", value = False, reason = f"unique(K) < 3")
        if not get("Active"):
            continue   # if not active
        
        # loop through the two selected dates
        for days in low_2_days:
            TTM = days / 365.0
            # if days > 100: break
            
            # select TTM data (only the given days til expiration)
            od_TTM = od_ticker[od_ticker["days"] == days]
            
            # interpolate the interest rate and forward rate
            r = vp.interpolated_ZCY(ZCY_date, days)
            F = vp.interpolated_FW(FW_ticker, days)
            
            # IV = T_data["IV"]["impl_volatility"] #smile, would need to be interpolated            
            # K = T_data["od"]["strike_price"]
            # IV = T_data["od"]["impl_volatility"]
            # cp_flag = T_data["od"]["cp_flag"]
            # option_prices = vp.BSM(F, K, days, IV, r, cp_flag)
        
            # original slice where days match (idx: od_TTM -> od)
            idx = od_TTM.index

            # modify the original DataFrame in place
            od.loc[idx, "IV_option_price"] = od_TTM.apply(
                lambda row: vp.BSM(
                    F,
                    row["strike_price"],
                    TTM,
                    row["impl_volatility"],
                    r,
                    row["cp_flag"]
                ),
                axis=1
            )
            od.loc[idx, "moneyness"] = np.log10(F / od_TTM["strike_price"])
            
            
    # Print progress
    elapsed_time = time.time() - start_time
    sys.stdout.write(f"\rRunning date: {current_date} | Tickers: {len(tickers_date)} | Time elapsed: {elapsed_time:.2f} seconds")
    sys.stdout.flush()

0.05720860600000001
0.056719222
0.05720860600000001
0.056719222
0.05720860600000001
0.056719222
0.05720860600000001
0.056719222
0.05720860600000001
0.056719222
0.05720860600000001
0.056719222
Running date: 1996-01-04 00:00:00 | Tickers: 6 | Time elapsed: 0.11 seconds0.05722388228571428
0.05671365028571428
0.05722388228571428
0.05671365028571428
0.05722388228571428
0.05671365028571428
0.05722388228571428
0.05671365028571428
0.05722388228571428
0.05671365028571428
0.05722388228571428
0.05671365028571428
Running date: 1996-01-05 00:00:00 | Tickers: 6 | Time elapsed: 0.22 seconds0.05697607923076923
0.05685371777777778
0.05697607923076923
0.05685371777777778
0.05697607923076923
0.05685371777777778
0.05697607923076923
0.05685371777777778
0.05697607923076923
0.05685371777777778
0.05697607923076923
0.05685371777777778
Running date: 1996-01-08 00:00:00 | Tickers: 6 | Time elapsed: 0.34 seconds0.05726993400000001
0.05685795888888889
0.05726993400000001
0.05685795888888889
0.05726993400000001
0.0

KeyboardInterrupt: 

In [46]:
# od_TTM
# F

In [48]:
od_TTM

,secid,date,exdate,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,ticker,n_trading_day,days,spread,IV_option_price,moneyness
2851974,109764,2003-02-28,2003-04-19,C,510.0,0.30,0.45,0,0.247765,OEX,1800,50,0.15,NaN,NaN
2851975,109764,2003-02-28,2003-04-19,C,515.0,0.05,0.35,0,0.235862,OEX,1800,50,0.30,NaN,NaN
2851980,109764,2003-02-28,2003-04-19,C,500.0,0.35,0.65,0,0.236495,OEX,1800,50,0.30,NaN,NaN
2852009,109764,2003-02-28,2003-04-19,P,510.0,85.20,87.20,0,0.317292,OEX,1800,50,2.00,NaN,NaN
2852010,109764,2003-02-28,2003-04-19,P,515.0,90.20,92.20,0,0.330611,OEX,1800,50,2.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2852205,109764,2003-02-28,2003-04-19,P,470.0,47.80,49.30,0,0.270652,OEX,1800,50,1.50,NaN,NaN
2852206,109764,2003-02-28,2003-04-19,P,475.0,51.90,53.90,0,0.271504,OEX,1800,50,2.00,NaN,NaN
2852207,109764,2003-02-28,2003-04-19,P,480.0,56.50,58.50,0,0.276923,OEX,1800,50,2.00,NaN,NaN
2852208,109764,2003-02-28,2003-04-19,P,490.0,65.80,67.80,0,0.285431,OEX,1800,50,2.00,NaN,NaN


In [47]:
F

425.066304

In [49]:
# summary_dly_df